### Chamada de LLM para Clusterização de textos 
Este notebook é parte de um experimento que constrói um modelo de regressão para predição de preço de medicamentos para farmácia 
de manipulação.
Nesta etapa, vamos criar um dataframe com princípios ativos de medicamento.
Os princípos ativos estão descritos em Linguagem Natural. Não podemos afirmar que a descrição está com diferenças extremas, mas existem casos que precisam ser tratados, como por exemplo:
ALECRIM
ALECRIM ES
ALECRIM EXT SECO
ALECRIM EXTRATO SECO
Hipoteticamente, assumimos que as 4 descrições se referem a um único princípio ativo então os 4 casos vão estar contidos em um cluster de nome ALECRIM, por exemplo.

Este exemplo acima não explica todos os motivos pelo qual a clusterização foi feita, além dele temos medicamentos com nomes escritos com s OU z, terminados com m OU n, estre outros.
Os resultados dessa clusterização podem ser verificados no PowerBI / PDF de Análise Exploratória que acompanha este trabalho.


In [1]:
!pip install openai

In [ ]:
from io import StringIO

import openai
import pandas as pd
from IPython.core.display import display, HTML

/tmp/ipykernel_198202/456750120.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
# esta seção utiliza a chave para a conta da openAi que será utilizada para rodar o prompt de clusterização
# infelizmente por ser uma chave privada, não estou compartilhando com o leitor
key="aqui você cola a sua chave da OpenAI"
client = openai.OpenAI(api_key=key)

In [ ]:
# lemos os princípios ativos salvos no outro notebook
df = pd.read_excel('./principios_ativos.xlsx', index_col=False)

In [ ]:
# corrigimos um evento encontrado que precisou de tratamento, que foi a vírgula no meio de nomes de medicamentos
# essa solução foi a mais rápida para o tempo disponível para realizar todo o experimento
df['principio_ativo_tratado'] = df['principio_ativo'].str.replace(',', '.')

In [ ]:
#verificamos se a limpeza funcionou, vide linha 351 entre outras
display(HTML(df.to_html()))

,Unnamed: 0,principio_ativo,quantidade,principio_ativo_tratado
0,0,17 BETA ESTRADIOL BASE,2,17 BETA ESTRADIOL BASE
1,1,25 HIDROXI D,1,25 HIDROXI D
2,2,25(OH)VITAMINA D3,1,25(OH)VITAMINA D3
3,3,5 HTP,4,5 HTP
4,4,5 L HIDROXITRIPTOFANO,25,5 L HIDROXITRIPTOFANO
5,5,5 MTHF METHYLFOLATE 1:10,4,5 MTHF METHYLFOLATE 1:10
6,6,5-HTP,67,5-HTP
7,7,5HTP,50,5HTP
8,8,AAKG,2,AAKG
9,9,ABACATEIRO,1,ABACATEIRO


In [ ]:
CONTENT_INPUT = """Eu tenho uma lista com quase dois mil medicamentos, o nome de um medicamento é muito sensível,
 mas nessa lista eu tenho um mesmo medicamento escrito de diversas formas.
 Eu vou te dar primeiro alguns exemplos de equivalencia, e depois mais tarde te mando a lista e o resultado esperado
 que eu quero que voce gere: medicamentos com o sufixo ext seco e extrato seco e es , esse sufixos são a mesma coisa,
o nome do medicamento com '-' ou sem '-' se refere a mesma coisa,
voce pode sugerir sua abordagem para padronizar os nomes também. Pode corrigir grafias erradas também para clusterizar,
se voce tiver certeza. Vou te passar a lista em seguida com cada principio_ativo separado por '|', 
e quando eu te passar, quero a resposta assim a cada linha,
um linha para cada principio ativo : 'nome principio ativo recebido','nome clusterizado',
retorne somente a seguine estrutura sem descrever o que você processou: nome do principio_ativo original, nome do cluster.
Atenção, retorne apenas essa tupla a cada linha sem mais nada."""

In [ ]:
# essa função chama a API com o LLM
def call_openai(content, message):
    response = client.chat.completions.create(
        model="gpt-4o",
        max_tokens=16384,
        temperature=0.2,
        messages=[{"role": "system", "content": content}, {"role": "user", "content": message}]
    )

    return response.choices[0].message.content

In [ ]:
#precisamos separar a chamada em batch porque na primeira tentativa o lote inteiro não foi totalmente clusterizado de uma só vez
def batch_process(df, column, batch_size, content):
  list_data = df[column].tolist()
  openai_return = []
  
  for i in range(0, len(list_data), batch_size):
    batch = list_data[i:i + batch_size]
    stringfy_batch = '|'.join(batch)
    openai_return.append(call_openai(content, stringfy_batch))
  
  return openai_return

In [ ]:
# mandamos para clusterizar a cada 1000 principios ativos, neste contexto então a batch roda 2 vezes
batchs = batch_process(df, "principio_ativo_tratado", 1000, CONTENT_INPUT)

In [ ]:
#juntamos o resultado
unify_results = '\n'.join(batchs)

In [ ]:
#mandarmos o resutado para um dataframe pandas
arquivo = StringIO(unify_results)
clusters_df = pd.read_csv(arquivo, header=None, names=["original", "cluster"])

In [ ]:
#salvamos o cluster para ser usado no outro notebook
clusters_df.to_excel('clusters.xlsx')